In [765]:
import numpy as np
import pandas as pd

In [766]:
df = pd.read_excel("base_indicadores.xlsx", sheet_name="indicadores", engine="openpyxl")
df.columns = df.columns.str.lower()

In [767]:
df.loc[df["indicador"] == "cascada_ppto", "topico"] = "cascada"
df.loc[df["indicador"] == "cascada_ppto", "indicador"] = "ppto"

In [768]:
import unicodedata

def remover_acentos(texto):
    if isinstance(texto, str):
        nfkd = unicodedata.normalize("NFKD", texto)
        return "".join([char for char in nfkd if not unicodedata.combining(char)])
    return texto

# Tratando textos
df["abertura"] = df["abertura"].apply(remover_acentos)
df["abertura"] = df["abertura"].str.lower()
df["abertura"] = df["abertura"].str.replace(" ", "_")
df["abertura"] = df["abertura"].str.replace("-", "_")
df["abertura"] = df["abertura"].str.replace("(", "")
df["abertura"] = df["abertura"].str.replace(")", "")
df["abertura"] = df["abertura"].str.replace("/", "_")

df["topico"] = df["topico"].str.replace(" ", "_") # Off Balance

In [769]:
df[df["abertura"] == "rorwa"]

,topico,indicador,abertura,especial,prospera,select,pj,corporate,scib,private,consumer,total


In [770]:
segmentos = ["especial", "prospera", "select", "pj", "corporate", "scib", "private", "consumer"]
df.loc[df["indicador"] == "spread", segmentos] = (df.loc[df["indicador"] == "spread", segmentos] * 100).round(2)
df.loc[df["abertura"].isin(["rorwa", "ppto_taxa_impositiva", "ppto_eficiencia"]), segmentos] = df.loc[df["abertura"].isin(["rorwa", "ppto_taxa_impositiva", "ppto_eficiencia"]), segmentos] * 100

In [771]:
# calculando o atingimento do cascada
indicadores_atingimento = df.loc[(df["indicador"] == "cascada") & (~df["abertura"].isin(["rwa", "rorwa"]))]["abertura"].unique()

realizado = df.loc[(df["indicador"] == "cascada") & (~df["abertura"].isin(["rwa", "rorwa"]))][segmentos + ['total']].reset_index(drop=True)
ppto = df.loc[(df["indicador"] == "ppto")][segmentos + ['total']].reset_index(drop=True)

resultado_divisao = (realizado / ppto * 100).fillna(0)

resultado_divisao["topico"] = "cascada"
resultado_divisao["indicador"] = "atingimento"
resultado_divisao["abertura"] = [f"atingimento_{i}" for i in indicadores_atingimento]

df = pd.concat([df, resultado_divisao[["topico", "indicador", "abertura"] + segmentos + ['total']]], ignore_index=True)

In [772]:
# calculando a taxa impositiva, eficiência e rorwa

mob = df.loc[(df["indicador"] == "cascada") & (df["abertura"].isin(["mob"]))][segmentos + ['total']].reset_index(drop=True).fillna(0)
gastos = df.loc[(df["indicador"] == "cascada") & (df["abertura"].isin(["total_gastos"]))][segmentos + ['total']].reset_index(drop=True).fillna(0)

bai = df.loc[(df["indicador"] == "cascada") & (df["abertura"].isin(["bai"]))][segmentos + ['total']].reset_index(drop=True).fillna(0)
impostos = df.loc[(df["indicador"] == "cascada") & (df["abertura"].isin(["impostos"]))][segmentos + ['total']].reset_index(drop=True).fillna(0)

bdi = df.loc[(df["indicador"] == "cascada") & (df["abertura"].isin(["bdi"]))][segmentos + ['total']].reset_index(drop=True).fillna(0)
rwa = df.loc[(df["indicador"] == "cascada") & (df["abertura"].isin(["rwa"]))][segmentos + ['total']].reset_index(drop=True).fillna(0)

eficiencia = round(-gastos / mob * 100, 1)
eficiencia["abertura"] = "eficiencia"
taxa_impositiva = round(-impostos / bai * 100, 1)
taxa_impositiva["abertura"] = "taxa_impositiva"
rorwa = round(bdi / rwa * 100, 1)
rorwa["abertura"] = "rorwa"

df_aux = pd.concat([eficiencia, taxa_impositiva, rorwa], ignore_index=True)
df_aux["indicador"] = "cascada"
df_aux["topico"] = "cascada"

df_aux = df_aux[["topico", "indicador", "abertura"] + segmentos + ['total']]
df = pd.concat([df, df_aux], ignore_index=True)

In [773]:
gastos

,especial,prospera,select,pj,corporate,scib,private,consumer,total
0,-5761.0,-453.0,-7507.0,-6565.0,-742.0,-2420.0,-520.0,-1148.0,-26129.0


In [774]:
bdi

,especial,prospera,select,pj,corporate,scib,private,consumer,total
0,-1735.0,176.0,2328.0,4675.0,3315.0,4518.0,388.0,1209.0,15236.0


In [775]:
rwa

,especial,prospera,select,pj,corporate,scib,private,consumer,total
0,41286.86,1143.07,59090.22,80276.0,99616.0,122660.0,6890.0,70232.0,600644.0


In [776]:
176.0 / 1143.07

0.15397132284112086

In [777]:
rorwa

,especial,prospera,select,pj,corporate,scib,private,consumer,total,abertura
0,-4.2,15.4,3.9,5.8,3.3,3.7,5.6,1.7,2.5,rorwa


In [778]:
for col in df.select_dtypes(include=['number']).columns:
    df[col] = df[col].round(1)

In [779]:
df[df["topico"] == "cascada"]

,topico,indicador,abertura,especial,prospera,select,pj,corporate,scib,private,consumer,total
149,cascada,cascada,mob,14768.0,1269.0,18966.0,21408.0,6376.0,9229.0,1130.0,7218.0,81922.0
150,cascada,cascada,pdd,-9698.0,-412.0,-6047.0,-5900.0,-579.0,-109.0,-4.0,-3751.0,-26529.0
151,cascada,cascada,mol,5070.0,857.0,12919.0,15508.0,5797.0,9120.0,1126.0,3467.0,55393.0
152,cascada,cascada,oryp,-972.0,-70.0,-834.0,-1073.0,-20.0,-19.0,-2.0,-307.0,-3107.0
153,cascada,cascada,demais_ativos,-185.0,-1.0,-74.0,-11.0,0.0,0.0,0.0,0.0,-314.0
154,cascada,cascada,total_gastos,-5761.0,-453.0,-7507.0,-6565.0,-742.0,-2420.0,-520.0,-1148.0,-26129.0
155,cascada,cascada,bai,-1848.0,333.0,4504.0,7859.0,5035.0,6681.0,604.0,2012.0,25843.0
156,cascada,cascada,impostos,113.0,-157.0,-2176.0,-3184.0,-1720.0,-2163.0,-216.0,-803.0,-10607.0
157,cascada,cascada,bdi,-1735.0,176.0,2328.0,4675.0,3315.0,4518.0,388.0,1209.0,15236.0
158,cascada,cascada,rwa,41286.9,1143.1,59090.2,80276.0,99616.0,122660.0,6890.0,70232.0,600644.0


In [780]:
# df_melted = df.melt(id_vars=["Topico", "Indicador", "Abertura"], var_name="Segmento", value_name="Valor")

In [781]:
aberturas_segmento = {
    "credito": {
        "especial": ['cheque especial', 'cartões', 'cp', 'consignado', 'hipotecas', 'autos', 'demais'],
        "prospera": ['cheque especial', 'cartões', 'cp', 'consignado', 'hipotecas', 'autos', 'microcrédito', 'demais'],
        "select": ['cheque especial', 'cartões', 'cp', 'consignado', 'hipotecas', 'autos', 'agro', 'comex', 'demais'],
        "pj": ['cheque emp/adp', 'cartões', 'hipotecas', 'autos', 'agro', 'capital de giro', 'comex', 'conta garantida', 'confirming', 'internegócios', 'demais'],
        "corporate": ['hipotecas', 'autos', 'agro', 'capital de giro', 'comex', 'conta garantida', 'confirming', 'internegócios', 'demais'],
        "scib": ['cartões', 'hipotecas', 'autos', 'agro', 'capital de giro', 'comex', 'confirming', 'internegócios', 'demais'],
        "private": ['cartões', 'cp', 'hipotecas', 'autos', 'agro', 'capital de giro', 'comex', 'internegócios', 'demais'],
        "consumer": ['cp', 'autos', 'demais']
    },

    "captacoes": {
        "especial": ['dav', 'contamax', 'cdb', 'poupança', 'letras', 'demais'],
        "prospera": ['dav', 'contamax', 'cdb', 'poupança', 'letras', 'demais'],
        "select": ['dav', 'contamax', 'cdb', 'poupança', 'letras', 'captações comex', 'demais'],
        "pj": ['dav', 'contamax', 'cdb', 'poupança', 'letras', 'captações comex', 'time deposit', 'demais'],
        "corporate": ['dav', 'contamax', 'cdb', 'letras', 'captações comex', 'time deposit', 'demais'],
        "scib": ['dav', 'contamax', 'cdb', 'letras', 'captações comex', 'time deposit', 'lf', 'demais'],
        "private": ['dav', 'contamax', 'cdb', 'letras', 'captações comex', 'coe', 'demais'],
        "consumer": []
    },

    "comissoes": {
        "especial": ['cartões', 'seguros open', 'seguros related', 'tarifas c/c', 'capitalização', 'consórcio', 'esfera', 'fidelização inss', 'tecban', 'demais'],
        "prospera": ['cartões', 'seguros open', 'seguros related', 'tarifas c/c', 'capitalização', 'consórcio', 'esfera', 'fidelização inss', 'tecban', 'demais'],
        "select": ['cartões', 'seguros open', 'seguros related', 'tarifas c/c', 'capitalização', 'consórcio', 'esfera', 'tecban', 'aaa', 'demais'],
        "pj": ['cartões', 'seguros open', 'seguros related', 'tarifas c/c', 'capitalização', 'consórcio', 'esfera', 'tecban', 'abertura conta', 'adquirência', 'fx', 'cash', 'demais'],
        "corporate": ['cartões', 'seguros open', 'seguros related', 'tarifas c/c', 'capitalização', 'consórcio', 'mercado de capitais', 'fiança', 'abertura conta', 'adquirência', 'fx', 'cash', 'demais'],
        "scib": ['mercado de capitais', 'fx', 'fiança', 'comex', 'cash', 'corretagem', 'demais'],
        "private": ['previdência', 'fundos', 'fx', 'mercado de capitais', 'comex', 'esfera', 'cartões', 'demais'],
        "consumer": ['seguros open', 'seguros related', 'tarifas de crédito', 'demais']
    }
}

In [782]:
for k in aberturas_segmento:
    for abertura in aberturas_segmento[k]:
        aberturas_segmento[k][abertura] = [i.replace(" ", "_") for i in aberturas_segmento[k][abertura]]
        aberturas_segmento[k][abertura] = [i.replace("/", "") for i in aberturas_segmento[k][abertura]]
        aberturas_segmento[k][abertura] = [remover_acentos(i) for i in aberturas_segmento[k][abertura]]

In [783]:
def calculate_spread(df, segmentos):
    new_spread = pd.DataFrame(
        np.array(
            df.loc[df["indicador"] == "margem"][segmentos].astype("float")) /
             np.array(df.loc[df["indicador"] == "carteira"][segmentos].astype("float")) * 100,
        columns=segmentos
    ).fillna(0)
    df.loc[df["indicador"] == "spread", segmentos] = np.array(new_spread)
    return df

In [784]:
def get_numbers(df, topico, indicador, segmento, aberturas_segmento):
    new_df = df.loc[(df["topico"] == topico) & (df["indicador"] == indicador), ["abertura", segmento]]
    if topico not in ["cascada"]:
        if segmento in ["especial", "prospera", "select"]:
            new_df["abertura"] = new_df["abertura"].apply(lambda x: "cheque_especial" if x == "cheque" else x)
        elif segmento == "PJ":
            new_df["abertura"] = new_df["abertura"].apply(lambda x: "cheque_empadp" if x == "cheque" else x)
        if segmento != "total":
            new_df["abertura"] = new_df["abertura"].apply(lambda x: x if x in aberturas_segmento[topico][segmento] else "demais")
    else:
        new_df = df.loc[(df["topico"] == topico) & (df["indicador"] == indicador), ["abertura", segmento]]
        
    return new_df.groupby(["abertura"])[segmento].sum().round(2).to_dict()

In [785]:
gastos

,especial,prospera,select,pj,corporate,scib,private,consumer,total
0,-5761.0,-453.0,-7507.0,-6565.0,-742.0,-2420.0,-520.0,-1148.0,-26129.0


In [786]:
df[df["abertura"] == "rorwa"]

,topico,indicador,abertura,especial,prospera,select,pj,corporate,scib,private,consumer,total
179,cascada,cascada,rorwa,-4.2,15.4,3.9,5.8,3.3,3.7,5.6,1.7,2.5


In [787]:
get_numbers(df, "cascada", "cascada", "prospera", aberturas_segmento)
# indicador = ['cascada', 'ppto', 'atingimento']
# ['carteira', 'spread', 'provisao', 'margem', 'rwa']

{'bai': 333.0,
 'bdi': 176.0,
 'demais_ativos': -1.0,
 'eficiencia': 35.7,
 'impostos': -157.0,
 'mob': 1269.0,
 'mol': 857.0,
 'oryp': -70.0,
 'pdd': -412.0,
 'rorwa': 15.4,
 'rwa': 1143.1,
 'taxa_impositiva': 47.1,
 'total_gastos': -453.0}

In [788]:
topicos = df["topico"].unique()
segmentos = ["especial", "prospera", "select", "pj", "corporate", "scib", "private", "consumer", "total"]
segmentos = [s.lower() for s in segmentos]
indicadores = df.groupby("topico")["indicador"].unique().to_dict()
indicadores_cascada = df.loc[df["topico"] == "cascada", "abertura"].unique()
# indicadores_cascada_ppto = df.loc[df["topico"] == "cascada_ppto", "abertura"].unique()

In [789]:
topicos

array(['credito', 'comissoes', 'captacoes', 'off_balance', 'cascada'],
      dtype=object)

In [790]:
indicadores

{'captacoes': array(['spread', 'carteira', 'margem'], dtype=object),
 'cascada': array(['cascada', 'ppto', 'atingimento'], dtype=object),
 'comissoes': array(['comissoes'], dtype=object),
 'credito': array(['carteira', 'spread', 'provisao', 'margem', 'rwa'], dtype=object),
 'off_balance': array(['spread', 'carteira', 'margem'], dtype=object)}

In [791]:
segmentos

['especial',
 'prospera',
 'select',
 'pj',
 'corporate',
 'scib',
 'private',
 'consumer',
 'total']

In [792]:
dict_indicadores = dict()
for segmento in segmentos:
    dict_indicadores[segmento] = {}
    for topico in topicos:
        if (topico == "off_balance") | (segmento == "total" and topico != "cascada"):
            continue
        dict_indicadores[segmento][topico] = {}
        for indicador in indicadores[topico]:
            dict_resultado = get_numbers(df, topico, indicador, segmento, aberturas_segmento)

            if topico == ["comissoes"]:
                dict_indicadores[segmento][topico] = dict_resultado
            else:
                dict_indicadores[segmento][topico][indicador] = dict_resultado

In [793]:
dict_indicadores

{'especial': {'credito': {'carteira': {'autos': 4970.0,
    'cartoes': 20157.0,
    'cheque_especial': 1747.0,
    'consignado': 38458.0,
    'cp': 6964.0,
    'demais': 67.0,
    'hipotecas': 11632.0},
   'spread': {'autos': 9.4,
    'cartoes': 17.0,
    'cheque_especial': 89.5,
    'consignado': 8.6,
    'cp': 22.3,
    'demais': 50.2,
    'hipotecas': 1.5},
   'provisao': {'autos': 380.0,
    'cartoes': 3317.0,
    'cheque_especial': 477.0,
    'consignado': 1957.0,
    'cp': 2917.0,
    'demais': 22.0,
    'hipotecas': 68.0},
   'margem': {'autos': 465.0,
    'cartoes': 3430.0,
    'cheque_especial': 1564.0,
    'consignado': 3294.0,
    'cp': 1555.0,
    'demais': 18.0,
    'hipotecas': 171.0},
   'rwa': {'autos': 3519.0,
    'cartoes': 7841.0,
    'cheque_especial': 1067.0,
    'consignado': 22553.0,
    'cp': 2496.0,
    'demais': 28.0,
    'hipotecas': 3579.0}},
  'comissoes': {'comissoes': {'capitalizacao': 448.0,
    'cartoes': 1654.0,
    'demais': 1822.0,
    'esfera': 33.0

In [794]:
# Cálculo do spread correto
for segmento in segmentos:
    for topico in dict_indicadores[segmento].keys():
        if topico in ["credito", "captacoes"]:
            for abertura in dict_indicadores[segmento][topico]["spread"]:
                try:
                    dict_indicadores[segmento][topico]["spread"][abertura] = round(dict_indicadores[segmento][topico]["margem"][abertura] / dict_indicadores[segmento][topico]["carteira"][abertura] * 100, 2)
                except:
                    dict_indicadores[segmento][topico]["spread"][abertura] = 0

In [795]:
dict_indicadores

{'especial': {'credito': {'carteira': {'autos': 4970.0,
    'cartoes': 20157.0,
    'cheque_especial': 1747.0,
    'consignado': 38458.0,
    'cp': 6964.0,
    'demais': 67.0,
    'hipotecas': 11632.0},
   'spread': {'autos': 9.36,
    'cartoes': 17.02,
    'cheque_especial': 89.52,
    'consignado': 8.57,
    'cp': 22.33,
    'demais': 26.87,
    'hipotecas': 1.47},
   'provisao': {'autos': 380.0,
    'cartoes': 3317.0,
    'cheque_especial': 477.0,
    'consignado': 1957.0,
    'cp': 2917.0,
    'demais': 22.0,
    'hipotecas': 68.0},
   'margem': {'autos': 465.0,
    'cartoes': 3430.0,
    'cheque_especial': 1564.0,
    'consignado': 3294.0,
    'cp': 1555.0,
    'demais': 18.0,
    'hipotecas': 171.0},
   'rwa': {'autos': 3519.0,
    'cartoes': 7841.0,
    'cheque_especial': 1067.0,
    'consignado': 22553.0,
    'cp': 2496.0,
    'demais': 28.0,
    'hipotecas': 3579.0}},
  'comissoes': {'comissoes': {'capitalizacao': 448.0,
    'cartoes': 1654.0,
    'demais': 1822.0,
    'esfera

In [796]:
import json

with open("dict_indicadores.json", "w", encoding="utf-8") as f:
    json.dump(dict_indicadores, f, indent=4, ensure_ascii=False)